# Dataset preprocessing

The goal of this notebook is to create a preprocessed kaggle dataset out of the competition dataset.  
For now, the preprocessing will be based on [this notebook](https://www.kaggle.com/code/vonmainstein/imu-tof).  
It consists of the following steps:
-   Set the appropriate dtypes (helps with RAM usage).
-   Impute missing feature values with forward, backward and then 0 filling.
-   Split the dataset into multiple cross validation folds.
-   Standardize feature values.
-   Pad/Truncate the sequences to the same length.  

> Note:  
> - Demographics data set will be ignored for now.  

## Imports

In [ ]:
import os
import json
from os.path import join
from itertools import repeat

import numpy as np
from numpy import ndarray
import pandas as pd
import plotly.express as px
from pandas import DataFrame as DF
from kagglehub import whoami, competition_download, dataset_upload

from config import *

## Data preprocessing

### Load dataset
Requires to be logged in if this notebook is not running on laggle, go to [your settings](https://www.kaggle.com/settings) to create an access token and put it in `~/.kaggle/`.

In [2]:
competition_dataset_path = competition_download(COMPETITION_HANDLE)
df = pd.read_csv(join(competition_dataset_path, "train.csv"), dtype=DATASET_DF_DTYPES)

### Impute missing data
Perform forward, backward and then 0 filling of NaN values.

In [3]:
feature_cols = list(set(df.columns) - set(META_DATA_COLUMNS))
# Missing ToF values are already imputed by -1 which is inconvinient since we want all missing values to be NaN.    
# So we replace them by NaN and then perform imputing.  
to_replace = {col: -1.0 for col in df.columns if col.startswith("tof")}

df[feature_cols] = (
    df
    .loc[:, feature_cols]
     # df.replace with np.nan sets dtype to floar64 so we set it back to float32
    .replace(to_replace, value=np.nan)
    .astype("float32")
    .groupby(df["sequence_id"], observed=True, as_index=False)
    .ffill()
    .groupby(df["sequence_id"], observed=True, as_index=False)
    .bfill()
    # In case there are only nan in the column in the sequence
    .fillna(0)
)

### One hot encode target values.

In [4]:
one_hot_target = pd.get_dummies(df["gesture"])
df[one_hot_target.columns] = one_hot_target
df

/tmp/ipykernel_13543/1507406683.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[one_hot_target.columns] = one_hot_target
/tmp/ipykernel_13543/1507406683.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[one_hot_target.columns] = one_hot_target
/tmp/ipykernel_13543/1507406683.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented 

,row_id,sequence_type,sequence_id,sequence_counter,subject,orientation,behavior,phase,gesture,acc_x,...,Neck - scratch,Text on phone,Wave hello,Write name in air,Write name on leg,Drink from bottle/cup,Pinch knee/leg skin,Pull air toward your face,Scratch knee/leg skin,Glasses on/off
0,SEQ_000007_000000,Target,SEQ_000007,0,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.683594,...,False,False,False,False,False,False,False,False,False,False
1,SEQ_000007_000001,Target,SEQ_000007,1,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.949219,...,False,False,False,False,False,False,False,False,False,False
2,SEQ_000007_000002,Target,SEQ_000007,2,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.722656,...,False,False,False,False,False,False,False,False,False,False
3,SEQ_000007_000003,Target,SEQ_000007,3,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.601562,...,False,False,False,False,False,False,False,False,False,False
4,SEQ_000007_000004,Target,SEQ_000007,4,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.566406,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574940,SEQ_065531_000048,Non-Target,SEQ_065531,48,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.503906,...,False,False,False,False,True,False,False,False,False,False
574941,SEQ_065531_000049,Non-Target,SEQ_065531,49,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.773438,...,False,False,False,False,True,False,False,False,False,False
574942,SEQ_065531_000050,Non-Target,SEQ_065531,50,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.082031,...,False,False,False,False,True,False,False,False,False,False
574943,SEQ_065531_000051,Non-Target,SEQ_065531,51,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.964844,...,False,False,False,False,True,False,False,False,False,False


### ToF data aggregation.
Time of Flight columns take most of the data, let's reduce their size by aggregating by mean for each Time of Flight sensor.

In [5]:
def agg_tof_cols_per_sensor(df:DF) -> DF:
    for tof_idx in range(1, 6):
        tof_name = f"tof_{tof_idx}"
        tof_cols = [f"{tof_name}_v{v_idx}" for v_idx in range(64)]
        if any(map(lambda col: col not in df.columns, tof_cols)):
            print(f"Some (or) all ToF {tof_idx} columns are not in the df. Maybe you already ran this cell?")
            continue
        df = (
            df
            # Need to use a dict otherwise the name of the col will be "tof_preffix" instead of the value it contains
            .assign(**{tof_name:df[tof_cols].mean(axis="columns")})
            .drop(columns=tof_cols)
        )
    return df

df = agg_tof_cols_per_sensor(df)
# Redifine feature_cols now that there are less of them.
feature_cols = list(set(df.columns) - set(META_DATA_COLUMNS) - set(df["gesture"].unique().tolist()))

df

,row_id,sequence_type,sequence_id,sequence_counter,subject,orientation,behavior,phase,gesture,acc_x,...,Drink from bottle/cup,Pinch knee/leg skin,Pull air toward your face,Scratch knee/leg skin,Glasses on/off,tof_1,tof_2,tof_3,tof_4,tof_5
0,SEQ_000007_000000,Target,SEQ_000007,0,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.683594,...,False,False,False,False,False,139.250000,117.109375,91.687500,123.359375,135.343750
1,SEQ_000007_000001,Target,SEQ_000007,1,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.949219,...,False,False,False,False,False,139.796875,119.671875,97.921875,124.406250,137.000000
2,SEQ_000007_000002,Target,SEQ_000007,2,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.722656,...,False,False,False,False,False,142.375000,128.359375,116.953125,125.687500,140.234375
3,SEQ_000007_000003,Target,SEQ_000007,3,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.601562,...,False,False,False,False,False,154.109375,142.093750,144.515625,149.078125,142.609375
4,SEQ_000007_000004,Target,SEQ_000007,4,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.566406,...,False,False,False,False,False,177.953125,149.453125,161.828125,163.765625,151.265625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574940,SEQ_065531_000048,Non-Target,SEQ_065531,48,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.503906,...,False,False,False,False,False,68.562500,67.750000,144.437500,74.062500,52.843750
574941,SEQ_065531_000049,Non-Target,SEQ_065531,49,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.773438,...,False,False,False,False,False,70.234375,66.656250,144.000000,70.406250,54.531250
574942,SEQ_065531_000050,Non-Target,SEQ_065531,50,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.082031,...,False,False,False,False,False,66.671875,62.906250,136.906250,70.109375,57.468750
574943,SEQ_065531_000051,Non-Target,SEQ_065531,51,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.964844,...,False,False,False,False,False,67.218750,64.500000,140.531250,75.609375,54.937500


### Split into folds

In [6]:
N_FOLDS = 5
VALIDATION_FRACTION = 0.2
def split_dataset(df:DF) -> tuple[DF, DF]:
    unique_sequences = df["sequence_id"].unique()
    validation_sequences = pd.Series(unique_sequences).sample(
        frac=VALIDATION_FRACTION, replace=False
    )

    validation_set = df[df["sequence_id"].isin(validation_sequences)]
    train_set = df[~df["sequence_id"].isin(validation_sequences)]

    return train_set, validation_set

folds = list(map(split_dataset, repeat(df, N_FOLDS)))

In [7]:
display(df.shape)
display(folds[0][0].shape)
display(folds[0][1].shape)

(574945, 44)

(460213, 44)

(114732, 44)

In [8]:
for train_df, validation_df in folds:
    print(train_df.shape, validation_df.shape)

(460213, 44) (114732, 44)
(460785, 44) (114160, 44)
(458476, 44) (116469, 44)
(460460, 44) (114485, 44)
(459761, 44) (115184, 44)


### Std norm
Standard scale the feature cols (should probably do something different for IMU cols).  
<!-- *Deprecated, std norm is now performed at dataset creation to avoid target leakage.*   -->

In [9]:
from itertools import starmap

def std_norm_dataset(train:DF, val:DF) -> tuple[DF, DF]:
    means = train[feature_cols].mean()
    std = train[feature_cols].std()
    train.loc[:, feature_cols] = (train[feature_cols] - means) / std
    val.loc[:, feature_cols] = (val[feature_cols] - means) / std
    return train, val

normed_folds = list(starmap(std_norm_dataset, folds))

Let's compare the train to validation mean/std skews.

In [10]:
pd.concat({
    "train": folds[0][1][feature_cols].agg(["mean", "std"]),
    "validation": folds[0][1][feature_cols].agg(["mean", "std"]),
})

rot_y     acc_x     thm_2     rot_w     rot_x     thm_1  \
train      mean -0.048616  0.034737 -0.012134  0.031899 -0.033015  0.000965   
           std   0.977685  1.010345  1.049974  0.997445  1.006909  1.016639   
validation mean -0.048616  0.034737 -0.012134  0.031899 -0.033015  0.000965   
           std   0.977685  1.010345  1.049974  0.997445  1.006909  1.016639   

                    thm_5     acc_y     tof_3     tof_4     tof_2     rot_z  \
train      mean  0.004610 -0.019937 -0.013850 -0.042127 -0.016541 -0.003062   
           std   1.022446  1.003688  1.003523  1.000151  0.998810  0.995238   
validation mean  0.004610 -0.019937 -0.013850 -0.042127 -0.016541 -0.003062   
           std   1.022446  1.003688  1.003523  1.000151  0.998810  0.995238   

                    acc_z     tof_1     thm_4     tof_5     thm_3  
train      mean  0.009306 -0.030251 -0.002652 -0.018148  0.024265  
           std   0.979159  1.007340  1.098138  0.995038  0.951666  
validation mean  0.009306 -0.030251 -0.002652 -0.018148  0.024265  
           std   0.979159  1.007340  1.098138  0.995038  0.951666

### Normalize sequences lengths.  
And turn the Dataframes into ndarrays.

#### Visualize histogram of sequences lengths.

Entire dataset sequences lengths.

In [11]:
px.histogram(
    (
        df
        .groupby("sequence_id", observed=True)
        .size()
    ),
    title="Sequence length frequency",
)

Second(to avoid always look at the first one) Train/validation split sequences lengths comparaison.

In [12]:
def get_set_sequences_lengths(set:DF, name:str) -> DF:
    return (
        set
        .groupby("sequence_id", observed=True)
        .size()
        .reset_index(name="length")
        .assign(set=name)
    )

full_se_lengths = pd.concat((
    get_set_sequences_lengths(folds[2][0], "Train"),
    get_set_sequences_lengths(folds[2][1], "Validation"),
))

fig = px.histogram(
    full_se_lengths,
    x="length",
    color="set",
    barmode="overlay",  # or 'group' if you want side-by-side bars
    nbins=50,           # adjust bin size if needed
    title="Sequence Length Distribution: Train vs Validation"
)

fig.update_traces(opacity=0.8)  # better visibility with overlay
fig.show()


In [13]:
for train, val in folds:
    print("train normed sequence len:", int(train.groupby("sequence_id", observed=True).size().quantile(SEQUENCE_NORMED_LEN_QUANTILE)))
    print("validation normed sequence len:", int(val.groupby("sequence_id", observed=True).size().quantile(SEQUENCE_NORMED_LEN_QUANTILE)))
    print()

train normed sequence len: 127
validation normed sequence len: 124

train normed sequence len: 129
validation normed sequence len: 124

train normed sequence len: 124
validation normed sequence len: 136

train normed sequence len: 126
validation normed sequence len: 127

train normed sequence len: 125
validation normed sequence len: 128



#### Sequence length norm implementation

In [14]:
from tqdm.notebook import tqdm

gesture_cols = df["gesture"].unique()

def length_normed_sequence_feat_arr(sequence: DF, normed_sequence_len: int) -> ndarray:
    features = (
        sequence
        .loc[:, feature_cols]
        .values
    )
    len_diff = abs(normed_sequence_len - len(features))
    if len(features) < normed_sequence_len:
        padded_features = np.pad(
            features,
            ((len_diff // 2 + len_diff % 2, len_diff // 2), (0, 0)),
        )
        return padded_features
    elif len(features) > normed_sequence_len:
        return features[len_diff // 2:-len_diff // 2]
    else:
        return features

def df_to_ndarray(df:DF, normed_sequence_len:int) -> np.ndarray:
    sequence_it = df.groupby("sequence_id", observed=True, as_index=False)
    x = np.empty(
        shape=(len(sequence_it), normed_sequence_len, len(feature_cols)),
        dtype="float32"
    )
    y = np.empty(
        shape=(len(sequence_it), df["gesture"].nunique()),
        dtype="float32"
    )
    for sequence_idx, (_, sequence) in tqdm(enumerate(sequence_it), total=len(sequence_it)):
        normed_seq_feat_arr = length_normed_sequence_feat_arr(sequence, normed_sequence_len)
        x[sequence_idx] = normed_seq_feat_arr
        # Take the first value as they are(or at least should be) all the same in a single sequence
        y[sequence_idx] = sequence[gesture_cols].iloc[0].values

    return x, y

def get_normed_seq_len(dataset:DF) -> int:
    return int(
        dataset
        .groupby("sequence_id", observed=True)
        .size()
        .quantile(SEQUENCE_NORMED_LEN_QUANTILE)
    )

def fold_dfs_to_ndarrays(train:DF, validation:DF) -> tuple[ndarray, ndarray, ndarray, ndarray]:
    """
    Returns:
        (train X, train Y, validation X, validation Y)
    """
    normed_sequence_len = get_normed_seq_len(train)
    return (
        *df_to_ndarray(train, normed_sequence_len),
        *df_to_ndarray(validation, normed_sequence_len),
    )

folds_arrs = list(starmap(fold_dfs_to_ndarrays, folds))

  0%|          | 0/6521 [00:00<?, ?it/s]

  0%|          | 0/1630 [00:00<?, ?it/s]

  0%|          | 0/6521 [00:00<?, ?it/s]

  0%|          | 0/1630 [00:00<?, ?it/s]

  0%|          | 0/6521 [00:00<?, ?it/s]

  0%|          | 0/1630 [00:00<?, ?it/s]

  0%|          | 0/6521 [00:00<?, ?it/s]

  0%|          | 0/1630 [00:00<?, ?it/s]

  0%|          | 0/6521 [00:00<?, ?it/s]

  0%|          | 0/1630 [00:00<?, ?it/s]

In [15]:
full_dataset_sequence_length_norm = get_normed_seq_len(df)
full_x, full_y = df_to_ndarray(df, full_dataset_sequence_length_norm)

  0%|          | 0/8151 [00:00<?, ?it/s]

## Create dataset

In [19]:
# Clean dataset directory if it already exists
! rm -rf preprocessed_dataset
# Create dataset direcory
! mkdir preprocessed_dataset
# Save folds
for fold_i, (train_x, train_y, val_x, val_y) in enumerate(folds_arrs):
    fold_dir_path = join("preprocessed_dataset", f"fold_{fold_i}")
    os.makedirs(fold_dir_path)
    # save features (X)
    np.save(join(fold_dir_path, "train_X.npy"), train_x, allow_pickle=False)
    # Save targets (Y)
    np.save(join(fold_dir_path, "train_Y.npy"), train_y, allow_pickle=False)
    # save features (X)
    np.save(join(fold_dir_path, "validation_X.npy"), val_x, allow_pickle=False)
    # Save targets (Y)
    np.save(join(fold_dir_path, "validation_Y.npy"), val_y, allow_pickle=False)
# Save full dataset
full_dataset_dir_path = "preprocessed_dataset/full_dataset"
os.makedirs(full_dataset_dir_path)
np.save(join(full_dataset_dir_path, "X.npy"), full_x, allow_pickle=False)
np.save(join(full_dataset_dir_path, "Y.npy"), full_y, allow_pickle=False)
# Save names of targets for inference
with open("preprocessed_dataset/targets_names.json", "w") as fp:
    json.dump(one_hot_target.columns.to_list(), fp, indent=1)

## Dataset upload
Optionally upload the dataset to kaggle.

In [18]:
if input("Do you want to upload the  dataset to kaggle?[yes/no]").lower() == "yes":
    # Updaload the dataset
    dataset_upload(
        join(whoami()["username"], "prepocessedyes-cmi-2025"),
        "preprocessed_dataset",
        version_notes="Preprocessed Child Mind Institue 2025 competition dataset."
    )
else:
    print("Dataset has not been uploaded.")

Kaggle credentials successfully validated.
Uploading Dataset https://www.kaggle.com/datasets/mauroabidalcarrer/prepocessed-cmi-2025 ...
Starting upload for file preprocessed_dataset/full_dataset/X.npy


Uploading: 100%|██████████| 70.4M/70.4M [00:45<00:00, 1.56MB/s]

Upload successful: preprocessed_dataset/full_dataset/X.npy (67MB)
Starting upload for file preprocessed_dataset/full_dataset/Y.npy



Uploading: 100%|██████████| 587k/587k [00:01<00:00, 568kB/s]  

Upload successful: preprocessed_dataset/full_dataset/Y.npy (573KB)
Starting upload for file preprocessed_dataset/fold_4/train_X.npy


...


Uploading: 100%|██████████| 14.1M/14.1M [00:05<00:00, 2.43MB/s]

Upload successful: preprocessed_dataset/fold_0/validation_X.npy (13MB)
Starting upload for file preprocessed_dataset/fold_0/train_Y.npy



Uploading: 100%|██████████| 470k/470k [00:00<00:00, 563kB/s] 

Upload successful: preprocessed_dataset/fold_0/train_Y.npy (459KB)


HTTPError: 400 Client Error: Bad Request for url: https://www.kaggle.com/api/v1/datasets/create/version/mauroabidalcarrer/prepocessed-cmi-2025